In [6]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from pathlib import Path
import json

secret_path = Path("resource").joinpath("secret.json")
secrets = json.loads(open(secret_path).read())
openai_api_key = secrets["OPENAI_API_KEY"]
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key)


chat_model.predict("안녕")

'안녕하세요! 무엇을 도와드릴까요?'

In [4]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

In [5]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from pathlib import Path
import json

prompt = "10자 이내로 설명"
secret_path = Path("resource").joinpath("secret.json")
secrets = json.loads(open(secret_path).read())
openai_api_key = secrets["OPENAI_API_KEY"]
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key)
conversation = ConversationChain(
    llm=chat_model, verbose=True, memory=ConversationBufferMemory()
)


chat_model.predict("비파가 뭐야?")

'비파는 중국의 전통 악기로, 13개의 줄이 있는 비슷한 형태의 도깨비 소리를 내는 악기입니다. 손가락으로 줄을 튕기면서 연주하며, 주로 중국 전통 음악에서 사용됩니다. 비파는 특히 중국 오페라에서 많이 사용되며, 그 외에도 다양한 음악 장르에서도 활용되고 있습니다.'

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from pathlib import Path
import json

template = """
10자 이내로 설명해야 함
Current conversation:
{history}
Human: {input}
AI Assistant:"""

secret_path = Path("resource").joinpath("secret.json")
secrets = json.loads(open(secret_path).read())
openai_api_key = secrets["OPENAI_API_KEY"]
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key)
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(
    prompt=PROMPT,
    llm=chat_model,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="AI Assistant"),
)


chat_model.predict("비파가 뭐야?")

'비파는 음악에서 사용되는 한 종류의 악기로, 일반적으로 소리를 내기 위해 손가락으로 줄을 누르고, 다른 손으로 줄을 치는 방식으로 연주됩니다. 주로 중국 전통 음악에서 사용되며, 사람들이 손가락으로 줄을 누르면서 다른 손으로 줄을 치는 독특한 연주 방식으로 유명합니다. 비파는 일반적으로 21개의 줄을 가지고 있으며, 소리는 줄의 길이와 두께에 따라 달라집니다. 비파는 주로 솔로 연주나 앙상블 연주에 사용되며, 중국 전통 음악의 특색적인 소리를 만들어냅니다.'

In [20]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferMemory,
    CombinedMemory,
    ConversationSummaryMemory,
)

secret_path = Path("resource").joinpath("secret.json")
secrets = json.loads(open(secret_path).read())
openai_api_key = secrets["OPENAI_API_KEY"]
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=openai_api_key)

conv_memory = ConversationBufferMemory(
    memory_key="chat_history_lines", input_key="input"
)

summary_memory = ConversationSummaryMemory(llm=chat_model, input_key="input")
# Combined
memory = CombinedMemory(memories=[conv_memory, summary_memory])
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Summary of conversation:
{history}
Current conversation:
{chat_history_lines}
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input", "chat_history_lines"],
    template=_DEFAULT_TEMPLATE,
)



conversation = ConversationChain(llm=chat_model, verbose=True, memory=memory, prompt=PROMPT)

chat_model.predict("비파가 뭐야?")